In [2]:
from main.AngleSmartAPI import AngleOne_Smart_API
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import time
api_key = "vhAupRK9"

token  = "IV7PPFHDE4RAWYS7OOXQIBLKTI"
pwd = "5689"
username = "AAAJ289396"
instance = AngleOne_Smart_API(api_key,username,pwd,token)
instance.connect()

exchange = "NSE"
symbol = "TATAMOTORS-EQ"
time_frame = "ONE_DAY"
start_date = "2024-03-10"
# end_date  = "2023-06-01"
# data = instance.get_data(exchange, symbol, time_frame, start_date, end_date)
# print(data)
final_data = pd.DataFrame()

# Start the loop to fetch data in 6-month intervals
current_date = datetime.strptime(start_date, "%Y-%m-%d")
today_date = datetime.today()

while current_date < today_date:
    next_date = current_date + relativedelta(months=3)
  
    end_date = min(next_date, today_date).strftime("%Y-%m-%d")

    # Fetch data for the current period
    data = instance.get_data(exchange, symbol, time_frame, current_date.strftime("%Y-%m-%d"), end_date)
    
    # Ensure data is a DataFrame and not empty
    if isinstance(data, pd.DataFrame) and not data.empty:
        final_data = pd.concat([final_data, data], ignore_index=True)
 
    # Move to the next period
    current_date = next_date  
    time.sleep(3)
final_data.reset_index(drop=True, inplace=True)
final_data

[I 250312 00:08:08 smartConnect:124] in pool


Successfully Connected 🟢


[I 250312 00:08:08 smartConnect:496] Search successful. Found 1 trading symbols for the given query:
    1. exchange: NSE, tradingsymbol: TATAMOTORS-EQ, symboltoken: 3456


Got Resources and Exchange Available 🙌🏻


KeyboardInterrupt: 

In [ ]:
import numpy as np


data = final_data
# short_sma = range(12,38,2)
# long_sma = range(26,89,4)

short_sma = range(16,17)
long_sma = range(66,67)
for sS in short_sma:
    start_date = end_date
    data["SHORT_SMA"] = data["Close"].rolling(window=sS).mean()
    for lS in long_sma:
        if sS>=lS:
            continue
        data["LONG_SMA"] = data["Close"].rolling(window=lS).mean()
        data.dropna(inplace = True)

        data["SIGNAL"] = np.where(data["SHORT_SMA"]>data["LONG_SMA"],"BUY","SELL")
        # print(data[130:225].head(50))
 
        isBuy = False
        bought_price = 0
        sold_price = 0
        count_trades = 0
        result_profit = []
        in_trade = False
        for i in range(len(data["SIGNAL"])):
            if isBuy == False and data["SIGNAL"].iloc[i] == "BUY":
                in_trade = True
                isBuy = True
                bought_price = data["Close"].iloc[i]

            elif isBuy == True and data["SIGNAL"].iloc[i] == "SELL" and in_trade == True:
                isBuy = False
                in_trade = False
                sold_price = data["Close"].iloc[i]
                pnl = ((sold_price-bought_price)/bought_price)*100
                result_profit.append(pnl)
                count_trades += 1
        print(f"Short SMA: {sS}, Long SMA: {lS}, Profit: {round(sum(result_profit),2)}, Trades Taken: {count_trades}")


        
   




Short SMA: 16, Long SMA: 66, Profit: 90.1, Trades Taken: 304


In [ ]:
import numpy as np
import pandas as pd  # Assuming pandas is used
data = final_data
short_sma = range(5, 21, 1)
long_sma = range(25, 50, 2)

result_df = pd.DataFrame(columns=["Short SMA", "Long SMA", "Total Profit (%)", "Trades Taken"])
for sS in short_sma:
    data["SHORT_SMA"] = data["Close"].rolling(window=sS).mean()
    for lS in long_sma:
        if sS >= lS:
            continue

        data["LONG_SMA"] = data["Close"].rolling(window=lS).mean()

        # Drop NaN rows only for this SMA combination, ensuring original data remains intact
        temp_data = data.dropna().copy()

        temp_data["SIGNAL"] = np.where(temp_data["SHORT_SMA"] > temp_data["LONG_SMA"], "BUY", "SELL")

        isBuy = False
        bought_price = 0
        result_profit = []
        count_trades = 0

        for i in range(len(temp_data["SIGNAL"])):
            if not isBuy and temp_data["SIGNAL"].iloc[i] == "BUY":
                isBuy = True
                bought_price = temp_data["Close"].iloc[i]

            elif isBuy and temp_data["SIGNAL"].iloc[i] == "SELL":
                isBuy = False
                sold_price = temp_data["Close"].iloc[i]
                pnl = ((sold_price - bought_price) / bought_price) * 100
                result_profit.append(pnl)
                count_trades += 1

        # print(f"Short SMA: {sS}, Long SMA: {lS}, Profit: {round(sum(result_profit), 2)}, Trades Taken: {count_trades}")
        result_df = pd.concat([
            result_df,
            pd.DataFrame({
                "Short SMA": [sS],
                "Long SMA": [lS],
                "Total Profit (%)": [round(sum(result_profit),2)],
                "Trades Taken": [count_trades],
                "Win Cofficient": [round(sum(result_profit),2)/count_trades]
            })
        ],ignore_index= True)

best_overall_return = result_df.loc[result_df["Total Profit (%)"].idxmax()]
best_win_cofficient = result_df.loc[result_df["Win Cofficient"].idxmax()]
print("Best Combination:")
print(best_overall_return)
print("<--->")
print(best_win_cofficient)

/var/folders/6c/q1fdmtjj36lfr0zwz07z_r0m0000gn/T/ipykernel_49717/522567586.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([


Best Combination:
Short SMA              19
Long SMA               29
Total Profit (%)     8.82
Trades Taken            4
Win Cofficient      2.205
Name: 184, dtype: object
<--->
Short SMA              19
Long SMA               29
Total Profit (%)     8.82
Trades Taken            4
Win Cofficient      2.205
Name: 184, dtype: object


In [ ]:
top_5_combinations = result_df.sort_values(by="Total Profit (%)", ascending=False).head(5)

print("Top 5 Best Combinations:")
print(top_5_combinations)

Top 5 Best Combinations:
    Short SMA Long SMA  Total Profit (%) Trades Taken
93         12       29            331.05           17
106        13       29            330.27           18
68         10       31            329.61           15
69         10       33            328.93           15
81         11       31            326.83           14


In [2]:
from main.AngleSmartAPI import AngleOne_Smart_API
from main.TestSMA_COMB import SMA_COMBO

api_key = "vhAupRK9"
token  = "IV7PPFHDE4RAWYS7OOXQIBLKTI"
pwd = "5689"
username = "AAAJ289396"

exchange = "NSE"
symbol = "HSCL-EQ"
time_frame = "FIFTEEN_MINUTE"
start_date = "2024-09-01"

instance = AngleOne_Smart_API(api_key,username,pwd,token)
instance.connect()
data = instance.get_FullData(exchange,symbol,time_frame,start_date)

short_sma = range(4,10,2)
long_sma = range(23,51,2)
SMAinstance = SMA_COMBO(data,short_sma,long_sma)

SMAinstance.getBestCombo()


[I 250312 00:12:01 smartConnect:124] in pool


Successfully Connected 🟢


[I 250312 00:12:02 smartConnect:496] Search successful. Found 1 trading symbols for the given query:
    1. exchange: NSE, tradingsymbol: HSCL-EQ, symboltoken: 14334


Got Resources and Exchange Available 🙌🏻


[I 250312 00:12:04 smartConnect:496] Search successful. Found 1 trading symbols for the given query:
    1. exchange: NSE, tradingsymbol: HSCL-EQ, symboltoken: 14334


KeyboardInterrupt: 